In [7]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from keras import backend as keras_backend
from keras.models import Sequential, load_model
from keras.layers import Dense, SpatialDropout2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, LeakyReLU,Flatten,Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [9]:
x = np.load("./preprocessed/x.npy")
y = np.load("./preprocessed/y.npy")

In [10]:
x[0].shape

(50, 2, 1)

In [4]:
def get_model():
  #forming model
  model = Sequential()

  #adding layers and forming the model
  model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(36,5,1)))
  model.add(MaxPooling2D(padding="same"))

  model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
  model.add(MaxPooling2D(padding="same"))
  model.add(Dropout(0.3))

  model.add(Flatten())

  model.add(Dense(512,activation="relu"))
  model.add(Dropout(0.4))

  model.add(Dense(512,activation="relu"))
  model.add(Dropout(0.4))

  model.add(Dense(10,activation="softmax"))
  
  #compiling
  model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

  return model

In [11]:
sum_train_loss = 0
sum_test_loss = 0
sum_train_acc = 0
sum_test_acc = 0

for i in range(10):
  model = get_model()
  
  x_train = np.concatenate([x[j] for j in [k for k in range(10) if k not in [i]]])
  y_train = np.concatenate([y[j] for j in [k for k in range(10) if k not in [i]]])

  x_test = x[i]
  y_test = y[i]

  model.fit(x_train, y_train, epochs=30, batch_size=50)

  #train and test loss and scores respectively
  train_eval = model.evaluate(x_train, y_train)
  test_eval = model.evaluate(x_test, y_test)

  print("Results for folder: ", i)
  print(train_eval)
  print(test_eval)

  sum_train_loss += train_eval[0]
  sum_test_loss += test_eval[0]
  sum_train_acc += train_eval[1]
  sum_test_acc += test_eval[1]

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
from sklearn.model_selection import KFold
import os

# Define the directory containing the preprocessed data
preprocessed_dir = "./preprocessed/"

# Define the number of folds
n_folds = 10

# Initialize cross-validation splitter
kf = KFold(n_splits=n_folds)

# Iterate over each fold
for fold_idx, (train_index, test_index) in enumerate(kf.split(range(n_folds))):
    print(f"Fold {fold_idx + 1}:")
    
    # Load training and testing data for the current fold
    train_data = []
    test_data = []
    for idx in range(n_folds):
        fold_dir = os.path.join(preprocessed_dir, f"fold{idx+1}")
        # Load data for the current fold
        # Assuming you have functions or methods to load data from each fold directory
        fold_train_data, fold_test_data = load_fold_data(fold_dir)
        if idx in train_index:
            train_data.extend(fold_train_data)
        elif idx in test_index:
            test_data.extend(fold_test_data)
    
    # Perform any necessary preprocessing or augmentation
    
    # Train and evaluate the model
    # Assuming you have functions or methods to train and evaluate the model
    train_model(train_data, test_data)
